## Import libraries and load data

In [101]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import pickle

%matplotlib inline

In [205]:
# Read Data

filepath = '/Users/kevinzhao/Kaggle/Recruit_Restaurant/inputs/'

air_reserve = pd.read_csv(filepath + 'air_reserve.csv')
hpg_reserve = pd.read_csv(filepath + 'hpg_reserve.csv')
air_store_info = pd.read_csv(filepath + 'air_store_info.csv')
hpg_store_info = pd.read_csv(filepath + 'hpg_store_info.csv')
air_visit_data = pd.read_csv(filepath + 'air_visit_data.csv')

store_id_relation = pd.read_csv(filepath + 'store_id_relation.csv')
sample_submission = pd.read_csv(filepath + 'sample_submission.csv')

## Merge dataframes

In [206]:
# Join Store reservation and store info dataframes 

air_data = air_reserve.merge(air_store_info, on='air_store_id', how='left')
hpg_data = hpg_reserve.merge(hpg_store_info, on='hpg_store_id', how='left')

In [207]:
# Add hpg ids to air data and air ids to hpg data

air_data = air_data.merge(store_id_relation, on ='air_store_id', how='left')
hpg_data = hpg_data.merge(store_id_relation, on ='hpg_store_id', how='left')

In [208]:
# Merge data sets 

air_data_full = air_data.merge(hpg_data, on =['air_store_id', 'reserve_datetime', 'visit_datetime'], how='left')
data_full = hpg_data.merge(air_data, on = ['hpg_store_id', 'reserve_datetime', 'visit_datetime'], how='outer')

In [209]:
air_data_full.shape, data_full.shape

((92427, 15), (2088053, 15))

Not all air store has a hpg store and vice versa...must combine both dataframes after imputation

In [210]:
print(air_data_full[(air_data_full.hpg_store_id_x.isnull()==True) & (air_data_full.hpg_store_id_y.isnull()==True)].shape)
print(air_data_full[(air_data_full.hpg_store_id_x.isnull()==True) & (air_data_full.hpg_store_id_y.isnull()==False)].shape)
print(air_data_full[(air_data_full.hpg_store_id_x.isnull()==False) & (air_data_full.hpg_store_id_y.isnull()==True)].shape)
print(air_data_full[(air_data_full.hpg_store_id_x.isnull()==False) & (air_data_full.hpg_store_id_y.isnull()==False)].shape)

(56114, 15)
(0, 15)
(31573, 15)
(4740, 15)


In [211]:
air_data_full[(air_data_full.hpg_store_id_x.isnull()==False) & (air_data_full.hpg_store_id_y.isnull()==False)][air_data_full.hpg_store_id_x!=air_data_full.hpg_store_id_y]

/Users/kevinzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,air_store_id,visit_datetime,reserve_datetime,reserve_visitors_x,air_genre_name,air_area_name,latitude_x,longitude_x,hpg_store_id_x,hpg_store_id_y,reserve_visitors_y,hpg_genre_name,hpg_area_name,latitude_y,longitude_y


hpg_store_id_y always matches hpg_store_id_x when the value is not NaN

In [212]:
# Rename columns
air_data_full = air_data_full.rename(columns= {'reserve_visitors_x' : 'air_reserve_visitors', 'latitude_x': 'latitude',
                                              'longitude_x':'longitude', 'reserve_visitors_y': 'hpg_reserve_visitors',
                                              'hpg_store_id_x':'hpg_store_id'})

# Drop redundant columns
air_data_full = air_data_full.drop(columns={'hpg_store_id_y', 'latitude_y', 'longitude_y'})

In [213]:
print(air_data_full[(air_data_full.air_area_name.isnull()==True) & (air_data_full.hpg_area_name.isnull()==True)].shape)
print(air_data_full[(air_data_full.air_area_name.isnull()==True) & (air_data_full.hpg_area_name.isnull()==False)].shape)
print(air_data_full[(air_data_full.air_area_name.isnull()==False) & (air_data_full.hpg_area_name.isnull()==True)].shape)
print(air_data_full[(air_data_full.air_area_name.isnull()==False) & (air_data_full.hpg_area_name.isnull()==False)].shape)

(0, 12)
(0, 12)
(90284, 12)
(2143, 12)


`air_area_name` is always present when `hpg_area_name` is present. Most data has `air_area_name` but not `hpg_area_name`.

In [214]:
#Drop hpg_area_name column because no information is added
air_data_full = air_data_full.drop(columns={'hpg_area_name'})

In [215]:
air_data_full.columns

Index(['air_store_id', 'visit_datetime', 'reserve_datetime',
       'air_reserve_visitors', 'air_genre_name', 'air_area_name', 'latitude',
       'longitude', 'hpg_store_id', 'hpg_reserve_visitors', 'hpg_genre_name'],
      dtype='object')

## Impute missing values

Impute missing `hpg_reserve_visitors` values as `air_reserve_visitors`

In [216]:
# air_data_full['hpg_reserve_visitors'] = air_data_full.apply(
#     lambda row: np.int64(row['air_reserve_visitors']) if math.isnan(row['hpg_reserve_visitors'])==True 
#     else np.int64(row['hpg_reserve_visitors']),
#     axis=1)

Impute missing `hpg_reserve_visitors` values as `air_reserve_visitors`

In [217]:
air_data_full['hpg_reserve_visitors'] = air_data_full.apply(
    lambda row: 0 if math.isnan(row['hpg_reserve_visitors'])==True 
    else np.int64(row['hpg_reserve_visitors']),
    axis=1)

In [218]:
#function to impute missing values from air to hpg
def fill_air_hpg(data, feature, is_string=False):
    """
    Fills missing values from one data set to another.
    :param data: dataframe to impute
    :param feature: feature on the dataframe to impute
    :param is_string: True if feature is string, false if feature is float
    """
    
    feature_x = feature + '_x'
    feature_y = feature + '_y'
    
    #if feature type is string like hpg_store_id, check for NaNs with type
    if is_string == True:
        data[feature_x] = data.apply(
            lambda row: row[feature_y] if type(row[feature_x])!=str else row[feature_x], axis=1)
        data[feature_y] = data.apply(
            lambda row: row[feature_x] if type(row[feature_y])!=str else row[feature_y], axis=1)
    #if feature type is float, check for NaNs using math.isnan()
    else:
        data[feature_x] = data.apply(
            lambda row: row[feature_y] if math.isnan(row[feature_x])==True else row[feature_x], axis=1)
        data[feature_y] = data.apply(
            lambda row: row[feature_x] if math.isnan(row[feature_y])==True else row[feature_y], axis=1)

In [219]:
#function to impute more general missing values from air to hpg
def fill_general(data, feature):
    """
    Fills missing genre and area name from one data set to another.
    :param data: dataframe to impute
    :param feature: feature on the dataframe to impute
    """
    
    air_feature = 'air_'+feature
    hpg_feature = 'hpg_'+feature
    
    data[air_feature] = data.apply(
        lambda row: row[hpg_feature] if type(row[air_feature])!=str else row[air_feature], axis=1)
    data[hpg_feature] = data.apply(
        lambda row: row[air_feature] if type(row[hpg_feature])!=str else row[hpg_feature], axis=1)

## Clean up features

Reformat from string to datetime

In [220]:
from datetime import datetime
from dateutil.parser import parse

In [221]:
air_data_full.loc[:,'visit_datetime'] = air_data_full.apply(
    lambda row: datetime.strptime(row['visit_datetime'], '%Y-%m-%d %H:%M:%S'), axis=1)
air_data_full.loc[:,'reserve_datetime'] = air_data_full.apply(
    lambda row: datetime.strptime(row['reserve_datetime'], '%Y-%m-%d %H:%M:%S'), axis=1)

## Feature Engineering

#### Date and Time

In [232]:
air_data_full.loc[:,'visit_year'] = air_data_full.apply(
    lambda row: row['visit_datetime'].year, axis=1)
air_data_full.loc[:,'visit_month'] = air_data_full.apply(
    lambda row: row['visit_datetime'].month, axis=1)
air_data_full.loc[:,'visit_day'] = air_data_full.apply(
    lambda row: row['visit_datetime'].day, axis=1)
air_data_full.loc[:,'visit_date'] = air_data_full.apply(
    lambda row: row['visit_datetime'].date(), axis=1)
air_data_full.loc[:,'visit_hour'] = air_data_full.apply(
    lambda row: row['visit_datetime'].hour, axis=1)

In [233]:
air_data_full.loc[:,'reserve_year'] = air_data_full.apply(
    lambda row: row['reserve_datetime'].year, axis=1)
air_data_full.loc[:,'reserve_month'] = air_data_full.apply(
    lambda row: row['reserve_datetime'].month, axis=1)
air_data_full.loc[:,'reserve_day'] = air_data_full.apply(
    lambda row: row['reserve_datetime'].day, axis=1)
air_data_full.loc[:,'reserve_date'] = air_data_full.apply(
    lambda row: row['reserve_datetime'].date(), axis=1)
air_data_full.loc[:,'reserve_hour'] = air_data_full.apply(
    lambda row: row['reserve_datetime'].hour, axis=1)

In [234]:
air_data_full.columns

Index(['air_store_id', 'hpg_store_id', 'visit_date', 'reserve_date',
       'visit_datetime', 'reserve_datetime', 'air_reserve_visitors',
       'hpg_reserve_visitors', 'air_genre_name', 'hpg_genre_name',
       'air_area_name', 'latitude', 'longitude', 'visit_year', 'visit_month',
       'visit_day', 'reserve_year', 'reserve_month', 'reserve_day',
       'visit_hour', 'reserve_hour'],
      dtype='object')

In [235]:
#reorder columns
air_columns = ['air_store_id', 'hpg_store_id','visit_date','reserve_date','visit_datetime','reserve_datetime', 'air_reserve_visitors',
               'hpg_reserve_visitors', 'air_genre_name', 'hpg_genre_name','air_area_name','latitude','longitude',
               'visit_year','visit_month','visit_day','reserve_year','reserve_month','reserve_day','visit_hour',
                 'reserve_hour']
air_data_full = air_data_full[air_columns]

In [236]:
air_data_full.head()

,air_store_id,hpg_store_id,visit_date,reserve_date,visit_datetime,reserve_datetime,air_reserve_visitors,hpg_reserve_visitors,air_genre_name,hpg_genre_name,...,latitude,longitude,visit_year,visit_month,visit_day,reserve_year,reserve_month,reserve_day,visit_hour,reserve_hour
0,air_877f79706adbfb06,NaN,2016-01-01,2016-01-01,2016-01-01 19:00:00,2016-01-01 16:00:00,1,0,Japanese food,NaN,...,35.658068,139.751599,2016,1,1,2016,1,1,19,16
1,air_db4b38ebe7a7ceff,hpg_bcecaa70222d94e5,2016-01-01,2016-01-01,2016-01-01 19:00:00,2016-01-01 19:00:00,3,0,Dining bar,NaN,...,34.676231,135.486059,2016,1,1,2016,1,1,19,19
2,air_db4b38ebe7a7ceff,hpg_bcecaa70222d94e5,2016-01-01,2016-01-01,2016-01-01 19:00:00,2016-01-01 19:00:00,6,0,Dining bar,NaN,...,34.676231,135.486059,2016,1,1,2016,1,1,19,19
3,air_877f79706adbfb06,NaN,2016-01-01,2016-01-01,2016-01-01 20:00:00,2016-01-01 16:00:00,2,0,Japanese food,NaN,...,35.658068,139.751599,2016,1,1,2016,1,1,20,16
4,air_db80363d35f10926,hpg_878cc70b1abc76f7,2016-01-01,2016-01-01,2016-01-01 20:00:00,2016-01-01 01:00:00,5,0,Dining bar,NaN,...,43.770635,142.364819,2016,1,1,2016,1,1,20,1


In [237]:
# pickle dataframe to load faster
# pickle.dump(air_data_full, open('air_data_full.p', 'wb'))
air_data_full = pickle.load(open('air_data_full.p', 'rb'))

#### Genre Name

First combine `air_genre_name` and `hpg_genre_name`

In [238]:
air_data_full.loc[:, 'genre_name'] = air_data_full.apply(
    lambda row: (row['air_genre_name'] + ' ' + row['hpg_genre_name']) 
    if (type(row['air_genre_name']) == str) and (type(row['hpg_genre_name']) == str)
    else row['air_genre_name'], axis=1)

In [239]:
#add feature for restaurants with Japanese in the name
air_data_full.loc[:,'Japanese_explicit'] = air_data_full.apply(
    lambda row: 1 if 'Japanese' in str(row.genre_name) else 0, axis=1)

In [240]:
#add feature for restaurants with Japanese food
air_data_full.loc[:,'Japanese'] = air_data_full.apply(
    lambda row: 1 if ('Japanese' in str(row.genre_name) or 'Izakaya' in str(row.genre_name)
                      or 'Sukiyaki' in str(row.genre_name) or 'Teppanyaki' in str(row.genre_name)
                      or 'Sushi' in str(row.genre_name)) 
                else 0, axis=1)

In [241]:
#add feature for Italian restaurants
air_data_full.loc[:,'Italian'] = air_data_full.apply(
    lambda row: 1 if ('Italian' in str(row.genre_name) or 'Pizza' in str(row.genre_name)) else 0, axis=1)

In [242]:
#add feature for French restaurants
air_data_full.loc[:,'French'] = air_data_full.apply(
    lambda row: 1 if 'French' in str(row.genre_name) else 0, axis=1)

In [243]:
#add feature for Korean restaurants
air_data_full.loc[:,'Korean'] = air_data_full.apply(
    lambda row: 1 if 'Korean' in str(row.genre_name) else 0, axis=1)

In [244]:
#add feature for restaurants with Chinese food
air_data_full.loc[:,'Chinese'] = air_data_full.apply(
    lambda row: 1 if ('Chinese' in str(row.genre_name) or 'Cantonese' in str(row.genre_name)
                      or 'Shanghai' in str(row.genre_name) or 'Sichuan' in str(row.genre_name)
                      or 'Dim Sum' in str(row.genre_name) or 'Taiwanese' in str(row.genre_name)) 
                else 0, axis=1)

In [245]:
#add feature for restaurants with Western food
air_data_full.loc[:,'Western'] = air_data_full.apply(
    lambda row: 1 if ('Western' in str(row.genre_name) or 'Italian' in str(row.genre_name)
                      or 'French' in str(row.genre_name) or 'Spanish' in str(row.genre_name)
                      or 'Spain' in str(row.genre_name) or 'Taiwanese' in str(row.genre_name)) 
                else 0, axis=1)

In [246]:
#add feature for restaurants with Chinese food
air_data_full.loc[:,'Asian'] = air_data_full.apply(
    lambda row: 1 if (row.Japanese == 1 or row.Korean == 1 or row.Chinese == 1) 
                else 0, axis=1)

In [247]:
air_data_full.genre_name.unique()

array(['Japanese food', 'Dining bar', 'Izakaya',
       'Okonomiyaki/Monja/Teppanyaki', 'Italian/French', 'Cafe/Sweets',
       'Dining bar Seafood', 'Yakiniku/Korean food', 'Western food',
       'Dining bar International cuisine', 'Izakaya Japanese style',
       'Bar/Cocktail', 'Izakaya Spain Bar/Italian Bar',
       'Yakiniku/Korean food Japanese style',
       'Dining bar Spain Bar/Italian Bar',
       'Italian/French Spain Bar/Italian Bar',
       'Japanese food Japanese style',
       'Japanese food Japanese food in general',
       'Bar/Cocktail International cuisine', 'Other',
       'Izakaya International cuisine', 'Italian/French Italian',
       'Western food Steak/Hamburger/Curry',
       'Yakiniku/Korean food Grilled meat', 'Creative cuisine',
       'Dining bar Italian', 'Izakaya Creation', 'Izakaya Seafood',
       'Karaoke/Party', 'Izakaya Creative Japanese food',
       'Bar/Cocktail Amusement bar', 'Bar/Cocktail Cafe',
       'Karaoke/Party Party',
       'Okonomiyak

In [248]:
# pickle.dump(air_data_full, open('air_data_full2.p', 'wb'))
air_data_full = pickle.load(open('air_data_full2.p', 'rb'))

#### Area Name

- Prefecture
- City

In [190]:
air_data_full.head()

,air_store_id,hpg_store_id,visit_datetime,reserve_datetime,air_reserve_visitors,hpg_reserve_visitors,air_genre_name,hpg_genre_name,air_area_name,latitude,...,reserve_hour,genre_name,Japanese_explicit,Italian,Korean,Chinese,French,Western,Japanese,Asian
0,air_877f79706adbfb06,NaN,2016-01-01 19:00:00,2016-01-01 16:00:00,1,1,Japanese food,NaN,Tōkyō-to Minato-ku Shibakōen,35.658068,...,16,Japanese food,1,0,0,0,0,0,1,1
1,air_db4b38ebe7a7ceff,hpg_bcecaa70222d94e5,2016-01-01 19:00:00,2016-01-01 19:00:00,3,3,Dining bar,NaN,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,...,19,Dining bar,0,0,0,0,0,0,0,0
2,air_db4b38ebe7a7ceff,hpg_bcecaa70222d94e5,2016-01-01 19:00:00,2016-01-01 19:00:00,6,6,Dining bar,NaN,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,...,19,Dining bar,0,0,0,0,0,0,0,0
3,air_877f79706adbfb06,NaN,2016-01-01 20:00:00,2016-01-01 16:00:00,2,2,Japanese food,NaN,Tōkyō-to Minato-ku Shibakōen,35.658068,...,16,Japanese food,1,0,0,0,0,0,1,1
4,air_db80363d35f10926,hpg_878cc70b1abc76f7,2016-01-01 20:00:00,2016-01-01 01:00:00,5,5,Dining bar,NaN,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,...,1,Dining bar,0,0,0,0,0,0,0,0


## Aggregate data

In [263]:
air_data_visits = air_data_full.loc[:,['air_store_id','hpg_store_id','visit_date','reserve_date',
                                      'air_reserve_visitors','hpg_reserve_visitors','visit_year']].groupby(
                                        ['air_store_id','visit_date'], as_index=False).agg('sum')

In [265]:
df_air_visit_data = air_visit_data.groupby(['air_store_id','visit_date'], as_index=False).agg('max')
df_air_visit_data

,air_store_id,visit_date,visitors
0,air_00a91d42b08b08d9,2016-07-01,35
1,air_00a91d42b08b08d9,2016-07-02,9
2,air_00a91d42b08b08d9,2016-07-04,20
3,air_00a91d42b08b08d9,2016-07-05,25
4,air_00a91d42b08b08d9,2016-07-06,29
5,air_00a91d42b08b08d9,2016-07-07,34
6,air_00a91d42b08b08d9,2016-07-08,42
7,air_00a91d42b08b08d9,2016-07-09,11
8,air_00a91d42b08b08d9,2016-07-11,25
9,air_00a91d42b08b08d9,2016-07-12,24


In [264]:
air_data_visits

,air_store_id,visit_date,air_reserve_visitors,hpg_reserve_visitors,visit_year
0,air_00a91d42b08b08d9,2016-10-31,2,2,2016
1,air_00a91d42b08b08d9,2016-12-05,9,0,2016
2,air_00a91d42b08b08d9,2016-12-14,18,0,2016
3,air_00a91d42b08b08d9,2016-12-17,2,0,2016
4,air_00a91d42b08b08d9,2016-12-20,4,0,2016
5,air_00a91d42b08b08d9,2017-02-18,9,9,2017
6,air_00a91d42b08b08d9,2017-02-23,12,12,2017
7,air_00a91d42b08b08d9,2017-03-01,3,4,2017
8,air_00a91d42b08b08d9,2017-03-14,4,0,2017
9,air_00a91d42b08b08d9,2017-03-21,3,0,2017


In [271]:
df_air = air_data_visits.merge(df_air_visit_data, on=['air_store_id','visit_date'], how='outer')

In [273]:
df_air[(df_air.air_reserve_visitors.isnull()==False)&(df_air.visitors.isnull()==False)]

,air_store_id,visit_date,air_reserve_visitors,hpg_reserve_visitors,visit_year,visitors


It looks like the restaurants from `air_visits` do not contain `air_reserve_visitors` and `hpg_reserve_visitors` data